In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [6]:
# images are 28x28
transformations = transforms.Compose([
    transforms.Resize(28),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

train_set = datasets.ImageFolder("/lab/vislab/DATA/MNIST/training/", transform = transformations)
val_set = datasets.ImageFolder("/lab/vislab/DATA/MNIST/testing/", transform = transformations)
print(train_set)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

val_loader = torch.utils.data.DataLoader(val_set, batch_size =32, shuffle=True)

# torch.nn.TripletMarginLoss(margin=1.0, p=2.0, eps=1e-06, swap=False, 
# size_average=None, reduce=None, reduction='mean')
# output = criterion(anchor, positive, negative)
criterion = MyTripletLoss() 

# Connector to GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print('Using device:', device)


# Sqaure size of each training image
img_size = 28

# Number of classes in the dataset
num_classes = 10

# Number of epochs to train for
num_epochs = 25

Dataset ImageFolder
    Number of datapoints: 58946
    Root location: /lab/vislab/DATA/MNIST/training/
    StandardTransform
Transform: Compose(
               Resize(size=28, interpolation=PIL.Image.BILINEAR)
               Grayscale(num_output_channels=1)
               ToTensor()
           )
Using device: cuda:1


In [7]:
class Model_(nn.Module):
    """Basic model for this set. Any changes or suggestions are welcome"""
    
    def __init__(self):
        super(Model_, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):  
        # conv 1  
        t = self.conv1(t)  
        t = F.relu(t)  
        t = F.max_pool2d(t, kernel_size=2, stride=2)   
        # conv 2  
        t = self.conv2(t)   
        t = F.relu(t)  
        t = F.max_pool2d(t, kernel_size=2, stride=2)   
        # fc1   
        t = t.reshape(-1, 12*4*4)  
        t = self.fc1(t)  
        t = F.relu(t)   
        # fc2  
        t = self.fc2(t)  
        t = F.relu(t)  
        # output  
        t = self.out(t)  
        # don't need softmax here since we'll use cross-entropy as activation.   
        return t

In [8]:
class MyTripletLoss(nn.Module):
    def __init__(self):
        super(MyTripletLoss, self).__init__()
        
    def forward(self, inputs, labels):
        # so inputs and labels are matrices
        losses = []
        batch_loss = 0.0
        # assume inputs and labels are same length
        
        for idx, anchor in enumerate(inputs): 
            positive = random.choice([image_ for i, image_ in enumerate(inputs) if labels[i] == labels[idx]])
            negative = random.choice([image_ for i, image_ in enumerate(inputs) if labels[i] != labels[idx]])
            
            # safety of deep copy
            a1 = anchor.clone().detach()
            a2 = anchor.clone().detach()

            dist1 = a1.sub(positive)    
            dist2 = a2.sub(negative)
            dist1 = dist1**2
            dist2 = dist2**2
            loss = max(dist1 - dist2 + 0.01)

            losses.append(loss)
        
        batch_loss = max(losses)
        return batch_loss

In [9]:
def train_model(model, optimizer, num_epochs):
    """Bodied function to train data to network"""
    
    train_loss = 0.0
    loss = 0.0    
    # sampler = torch.utils.data.RandomSampler(train_set, replacement=False, num_samples=1)
    # eval for train and test, use criterion and back propagation, specific towards
    # the type of loss that we want, contrastive
    for epoch in range(num_epochs):
        print("Epoch num: ", epoch)   
        model.train()
        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            output = model.forward(inputs)
          
            loss = criterion(output, labels)
            loss = Variable(loss, requires_grad = True)
            
            loss.backward()
            
            optimizer.step()
            
            train_loss += loss.item()*inputs.size(0)
            
        print("loss for the epoch is:", train_loss)
        model.eval()
        val_loss = 0 
        accuracy = 0
        counter = 0 
        with torch.no_grad():

            for inputs, labels in val_loader:
                # Move to device
                inputs, labels = inputs.to(device), labels.to(device)
                # Forward pass
                output = model.forward(inputs)
                # Calculate Loss
            
                valloss = criterion(inputs, labels)
                
                

                # ******* THIS PART WAS FOUND ONLINE *********
                #
                #
                # Add loss to the validation set's running loss
                val_loss += valloss.item()*inputs.size(0)

                # Since our model outputs a LogSoftmax, find the real 
                # percentages by reversing the log function
                output = torch.exp(output)
                # Get the top class of the output
                top_p, top_class = output.topk(1, dim=1)
                # See how many of the classes were correct?
                equals = top_class == labels.view(*top_class.shape)
                # Calculate the mean (get the accuracy for this batch)
                # and add it to the running accuracy for this epoch
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                # Print the progress of our evaluation
                counter += 1
                print(counter, "/", len(val_loader))
                #
                #
                #
                # **********************************************
            
    return model, total_loss

In [ ]:
# Run time script
model = Model_()
for param in model.parameters():
    param.requires_grad = False
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
print("Begin to train model ...")

result, loss = train_model(model, optimizer, num_epochs)
print("Done training model")

Begin to train model ...
Epoch num:  0


In [ ]:
# TODO set up some graphs and visualization
